In [2]:
import mlflow
mlflow.__version__

'3.5.0'

In [3]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import mlflow
import mlflow.sklearn


mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("yellow-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/3', creation_time=1761107679135, experiment_id='3', last_update_time=1761107679135, lifecycle_stage='active', name='yellow-taxi-experiment', tags={}>

In [4]:


# URL for March 2023 data
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"

# Read the Parquet file
df = pd.read_parquet(url)

# Print the number of records (rows)
print("Number of records loaded:", len(df))


Number of records loaded: 3403766


In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df = read_dataframe(url)

# Print the number of records
print("Number of records after filtering:", len(df))

Number of records after filtering: 3316216


In [7]:
# Prepare feature dictionary
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')

# Vectorize categorical features
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Target variable
y_train = df['duration']

# Train a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Print the model intercept
print("Model intercept:", lr.intercept_)

Model intercept: 24.77203445209766


In [ ]:
# Step 5: Log model
with mlflow.start_run():
    mlflow.sklearn.log_model(lr, "model")
    # mlflow.log_artifact("artifacts")

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("features", categorical)
    mlflow.log_metric("intercept", lr.intercept_)
      # Log model size as a metric
    mlflow.log_metric("model_size_bytes", model_size_bytes)

print("✅ Model logged successfully in MLflow!")


2025/10/22 05:15:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025/10/22 05:15:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run tasteful-sloth-179 at: http://localhost:5000/#/experiments/3/runs/546b7ec8ded4475db5fd5e873c35c2a8
🧪 View experiment at: http://localhost:5000/#/experiments/3
✅ Model logged successfully in MLflow!
